In [161]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import os
from os.path import join as oj
import numpy as np
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import pandas as pd
import data
import models
from colorama import Fore
import pickle as pkl
import viz
from style import *

outcome_def = 'y_consec_thresh'
out_dir = oj('/scratch/users/vision/abc', 'mar8_tf_order_1_2')
results = analyze_helper.load_results(out_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**baseline stats**

In [162]:
viz.print_metadata(acc=results.accuracy.max())

valid:		2066 aux+ / 7594 (0.272)
----------------------------------------
hotspots:	 349 aux+ /  349
short:		 212 aux+ / 4266 (0.950)
long:		  41 aux+ /   43 (0.953)
hard:		1482 aux+ / 2936 (0.505)
----------------------------------------
hard acc:			  0.763
total acc (no hotspots):	  0.874


**look at prediction metrics**

In [163]:
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['accuracy', 'balanced_accuracy'], ascending=False).head(100)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

,accuracy,balanced_accuracy,f1,precision,recall,roc_auc
model_type,,,,,,
svm_28_None=7_ros=1_h=0,0.763,0.771,0.765,0.765,0.78,0.771
svm_28_None=7_ros=1_h=1,0.761,0.769,0.763,0.764,0.775,0.769
rf_28_None=7_ros=1_h=0,0.76,0.771,0.76,0.771,0.767,0.848
svm_28_None=7_ros=1.2_h=0,0.76,0.768,0.77,0.746,0.811,0.768
svm_28_None=7_ros=1_h=-1,0.759,0.767,0.76,0.763,0.772,0.767
svm_16_None=7_ros=1_h=0,0.758,0.768,0.763,0.759,0.784,0.768
mlp2_16_None=7_ros=1_h=0,0.758,0.767,0.761,0.756,0.781,0.841
rf_28_None=7_ros=1.2_h=0,0.757,0.768,0.76,0.763,0.775,0.848
mlp2_16_None=7_ros=0.8_h=0,0.757,0.766,0.753,0.773,0.748,0.839


**look at feat importances**

In [ ]:
r = results
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
# r = r[r.accuracy > np.percentile(r.accuracy, 80)]

keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})
# r = r[r2.index.str.contains('ros')]
# r = r[r.index.str.contains('none')]
r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
r = rank(r)
# r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank

idxs = r.index
r.insert(0, 'acc', results.loc[idxs]['accuracy'])
r = r.sort_values('acc', ascending=False)
r = r.reindex(r.iloc[0].sort_values(ascending=True).index, axis=1) # sort cols by mean rank
subset = list(r.keys())
subset.remove('acc')
r.insert(1, 'num_feats', r.apply(lambda row: np.sum(~np.isnan(row.values)) - 1, axis=1))


def color_vals(val):
    colors = {
        1: cb, 2: cp, 3: cr,
    }
    if not val in colors:
        color = 'maroon'
    else:
        color = colors[val]
    return 'color: %s' % color

r = r[r.num_feats <= 5].head(10)

r.fillna("").style.applymap(color_vals, subset=subset)
# r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1, subset=subset) # rows differently colored